In [2]:
import numpy as np
import pandas as pd
import csv, os, random, sys
import scipy.stats

filename = '/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/meta_embeddings/glove.42d.300.txt.csv'
tes = pd.read_csv(filename,quoting=3,  error_bad_lines=False)

In [18]:
def _eval_sim(testfile,inp_emb):
    test, emb = np.empty(0), np.empty(0)
    testdrop = np.empty(0)
    spearman_corr = 0
    mean_vec = np.mean(np.asarray(list(inp_emb.values()) ), axis = 0)

    with open(testfile, 'r') as comp_test:
        tests_csv = csv.reader(comp_test)

        for line in tests_csv:
            try:
                word1, word2 = line[0], line[1]
            except:
                print (line)
            if word1 in inp_emb and word2 in inp_emb:
                wordvec_1 = inp_emb[word1]
                wordvec_2 = inp_emb[word2]
            else:
                continue
#             wordvec_1 = inp_emb[word1] if word1 in inp_emb else mean_vec
#             wordvec_2 = inp_emb[word2] if word2 in inp_emb else mean_vec
            try:
                test = np.append(test, float(line[2]))
            except:
                print(line)
            if np.any(wordvec_1) and np.any(wordvec_2):
                emb = np.append(emb, np.dot(wordvec_1, wordvec_2))
            else:
                emb = np.append(emb, 0)
    spearman_corr = scipy.stats.spearmanr(test, emb)[0]
        
    return spearman_corr

In [7]:
inp_emb = {}
for wordvec in tes.iterrows():
    word, vec = wordvec[1][0], wordvec[1][1:].tolist()
    vec = np.fromiter(map(float, vec[1:]), dtype = np.float32)
    norm = np.linalg.norm(vec)
    inp_emb[word] = vec/norm if (norm != 0) else [vec]


In [21]:
score_dict = {}
score_dict['score'] = 0
for root,dirs,files in os.walk('/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input'):
    files = [testfile for testfile in files if testfile[0]!='.']
    for testfile in files:
        f_path = root+'/'+testfile
        print (f_path)
#             score_dict[testfile[:-4].strip().lower().replace(" ", "_").replace("-", "_")] = _eval_sim(f_path, inp_emb)
        score_dict[testfile[:-4].strip().lower()] = _eval_sim(f_path, inp_emb)
        if  testfile != 'mc-30.csv':
            score_dict['score'] += _eval_sim(f_path, inp_emb)/(len(files)-1)

/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/mturk-287.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/verb-143.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/wordsim353.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/EN-MC-30.txt
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/mc-30.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/MEN_natural_full.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/MTURK-771.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/SimLex-999.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/SimVerb-3500.csv
/home/u15410/yizhen/emsemble/AutoencodedMetaEmbedding/VecShare/vecshare/Test_Input/stanford_rareword.csv


In [22]:
print (score_dict)

{'score': 0.5329003006906, 'mturk-287': 0.6440623687848714, 'verb-143': 0.3255615755411917, 'wordsim353': 0.6451454541425574, 'en-mc-30': 0.777345097277711, 'mc-30': 0.777345097277711, 'men_natural_full': 0.7411636189313144, 'mturk-771': 0.6790015729258259, 'simlex-999': 0.3738278855308309, 'simverb-3500': 0.22621189208829712, 'stanford_rareword': 0.38378324099279965}


In [27]:
pd.Series(score_dict).to_frame()

,0
en-mc-30,0.777345
mc-30,0.777345
men_natural_full,0.741164
mturk-287,0.644062
mturk-771,0.679002
score,0.532900
simlex-999,0.373828
simverb-3500,0.226212
stanford_rareword,0.383783
verb-143,0.325562
